# Datos Recompra Adquisicion

In [1]:
from snowflake.snowpark import Session
from configparser import ConfigParser
import snowflake.snowpark.functions as fn
import pandas as pd
import numpy as np

config = ConfigParser()
config.read('config.ini')

connection_parameters: dict[str, int | str] = {
    'user' : config.get('SNOWFLAKE', 'USER'),
    'password' : config.get('SNOWFLAKE', 'PASSWORD'),
    'account' : config.get('SNOWFLAKE', 'ACCOUNT'),
    'database' : config.get('SNOWFLAKE', 'DATABASE'),
    'warehouse' : config.get('SNOWFLAKE', 'WAREHOUSE'),
    'schema' : config.get('SNOWFLAKE', 'SCHEMA'),
    'role' : config.get('SNOWFLAKE', 'ROLE'),
}

session = Session.builder.configs(connection_parameters).create()

## Adquisicion general

In [2]:
from DP_transacciones_total import get_transacciones_dp_total
from general_primeras_compras import get_primeras_compras
from general_get_tiempo import get_tiempo

tiempo = get_tiempo(session)

transacciones_dp_total = get_transacciones_dp_total(session)

primeras_compras = get_primeras_compras(transacciones_dp_total)

primeras_compras_octubre = (
    primeras_compras
    .join(tiempo, on = 'FECHA')
    .filter(fn.col('ANIO_ALSEA') == 2023)
    .filter(fn.col('MES_ALSEA') == 10)
    .with_column_renamed('FECHA', 'FECHA_ADQUISICION')
    .select('EMAIL', 'FECHA_ADQUISICION')
)

datos_recompras_octubre = (
    primeras_compras_octubre
    .join(transacciones_dp_total, on = 'EMAIL')
    .filter(fn.col('FECHA') > fn.col('FECHA_ADQUISICION'))
    .group_by('EMAIL')
    .agg(
        fn.sum('VENTA').alias('VENTA'),
        fn.count_distinct('ORDER_ID').alias('TRANSACCIONES')
    )
    .order_by('TRANSACCIONES', ascending = False)
    .with_column('COHORT',
        fn.when(fn.col('TRANSACCIONES') > 3, 4)
        .otherwise(fn.col('TRANSACCIONES'))
    )
    .group_by('COHORT')
    .agg(
        fn.count_distinct('EMAIL').alias('USUARIOS'),
        fn.sum('VENTA').alias('VENTA'),
        fn.sum('TRANSACCIONES').alias('TRANSACCIONES')
    )
    .with_column('TICKET', fn.col('VENTA') / fn.col('TRANSACCIONES'))
    .with_column('FRECUENCIA', fn.col('TRANSACCIONES') / fn.col('USUARIOS'))
)

datos_recompras_octubre_pd = datos_recompras_octubre.to_pandas()
datos_recompras_octubre_pd.to_csv('datos_recompras_octubre.csv')
datos_recompras_octubre_pd

,COHORT,USUARIOS,VENTA,TRANSACCIONES,TICKET,FRECUENCIA
0,1,9162,2.296085e+06,9162,250.609561,1.000000
1,4,301,3.771071e+05,1548,243.609218,5.142857
2,3,524,3.713319e+05,1572,236.216219,3.000000
3,2,1904,8.992382e+05,3808,236.144483,2.000000


## Adquisicion 40% OFF

In [38]:
from DP_transacciones_total import get_transacciones_olo, get_transacciones_cloud

descuentos_40_OFF = (
    session
    .table('SEGMENT_EVENTS.ALOPERACIONES.TAFDESCUENTOSORDENES3')
    .filter(fn.col('FCIDDESCUENTO').isin([4493, 4494]))
    .select(['FCIDTIENDA', 'FITICKET'])
)

ordenes_40_OFF = (
    session
    .table('SEGMENT_EVENTS.ALOPERACIONES.TAFORDENES')
    .filter(fn.col('FIIDCLIENTE').is_not_null())
    .filter(fn.col('FIIDCLIENTE') != 0)
    .with_column('ID_CLIENTE', fn.concat(fn.col('FCIDTIENDA'), fn.col('FIIDCLIENTE')))
    .with_column('FECHA', fn.to_date('FDIDDIA'))
    .join(descuentos_40_OFF, on = ['FCIDTIENDA', 'FITICKET'], how = 'INNER')
)

clientes_40_OFF = (
    session
    .table('SEGMENT_EVENTS.ALMODCOMUN.TACCLIENTES')
    .with_column('ID_CLIENTE', fn.concat(fn.col('FIIDTIENDA'), fn.col('FIIDCLIENTE')))
    .join(ordenes_40_OFF, on = 'ID_CLIENTE')
)

transacciones_olo = get_transacciones_olo(session)

redenciones_olo_40_OFF = (
    transacciones_olo
    .join(clientes_40_OFF, on = ['ID_CLIENTE', 'FECHA'])
    .select(['EMAIL', 'FECHA'])
)

redenciones_cloud_40_OFF = (
    get_transacciones_cloud(session)
    .filter(fn.col('COUPONSCODE').like('%HELLO%'))
    .select(['EMAIL', 'FECHA'])
)

redenciones_totales_40_OFF = redenciones_cloud_40_OFF.union_all(redenciones_olo_40_OFF)

redenciones_octubre_40_OFF = (
    redenciones_totales_40_OFF
    .join(tiempo, on = 'FECHA')
    .filter(fn.col('ANIO_ALSEA') == 2023)
    .filter(fn.col('MES_ALSEA') == 10)
    .with_column_renamed('FECHA', 'FECHA_ADQUISICION')
    .select('EMAIL', 'FECHA_ADQUISICION')
)

redenciones_octubre_40_OFF.agg(fn.count_distinct('EMAIL')).show()

datos_recompras_octubre_40_OFF = (
    redenciones_octubre_40_OFF
    .join(transacciones_dp_total, on = 'EMAIL')
    .filter(fn.col('FECHA') > fn.col('FECHA_ADQUISICION'))
    .group_by('EMAIL')
    .agg(
        fn.sum('VENTA').alias('VENTA'),
        fn.count_distinct('ORDER_ID').alias('TRANSACCIONES')
    )
    .order_by('TRANSACCIONES', ascending = False)
    .with_column('COHORT',
        fn.when(fn.col('TRANSACCIONES') > 3, 4)
        .otherwise(fn.col('TRANSACCIONES'))
    )
    .group_by('COHORT')
    .agg(
        fn.count_distinct('EMAIL').alias('USUARIOS'),
        fn.sum('VENTA').alias('VENTA'),
        fn.sum('TRANSACCIONES').alias('TRANSACCIONES')
    )
    .with_column('TICKET', fn.col('VENTA') / fn.col('TRANSACCIONES'))
    .with_column('FRECUENCIA', fn.col('TRANSACCIONES') / fn.col('USUARIOS'))
)

datos_recompras_octubre_40_OFF_pd = datos_recompras_octubre_40_OFF.to_pandas()
datos_recompras_octubre_40_OFF_pd.to_csv('datos_recompras_octubre_40_OFF.csv')
datos_recompras_octubre_40_OFF_pd

---------------------------
|"COUNT(DISTINCT EMAIL)"  |
---------------------------
|11986                    |
---------------------------



,COHORT,USUARIOS,VENTA,TRANSACCIONES,TICKET,FRECUENCIA
0,1,802,249563.448276,802,311.176369,1.000000
1,4,48,287395.603448,264,1088.619710,5.500000
2,3,47,90176.034483,141,639.546344,3.000000
3,2,179,121819.698276,358,340.278487,2.000000


## Adquisicion GO199

In [4]:
from DP_transacciones_total import get_transacciones_olo, get_transacciones_cloud

descuentos_GO199 = (
    session
    .table('SEGMENT_EVENTS.ALOPERACIONES.TAFDESCUENTOSORDENES3')
    .filter(fn.col('FCIDDESCUENTO').isin([4647, 4648]))
    .select(['FCIDTIENDA', 'FITICKET'])
)

ordenes_GO199 = (
    session
    .table('SEGMENT_EVENTS.ALOPERACIONES.TAFORDENES')
    .filter(fn.col('FIIDCLIENTE').is_not_null())
    .filter(fn.col('FIIDCLIENTE') != 0)
    .with_column('ID_CLIENTE', fn.concat(fn.col('FCIDTIENDA'), fn.col('FIIDCLIENTE')))
    .with_column('FECHA', fn.to_date('FDIDDIA'))
    .join(descuentos_GO199, on = ['FCIDTIENDA', 'FITICKET'], how = 'INNER')
)

clientes_GO199 = (
    session
    .table('SEGMENT_EVENTS.ALMODCOMUN.TACCLIENTES')
    .with_column('ID_CLIENTE', fn.concat(fn.col('FIIDTIENDA'), fn.col('FIIDCLIENTE')))
    .join(ordenes_GO199, on = 'ID_CLIENTE')
)

transacciones_olo = get_transacciones_olo(session)

redenciones_olo_GO199 = (
    transacciones_olo
    .join(clientes_GO199, on = ['ID_CLIENTE', 'FECHA'])
    .select(['EMAIL', 'FECHA'])
)

redenciones_cloud_GO199 = (
    get_transacciones_cloud(session)
    .filter(fn.col('COUPONSCODE').like(r'%GO199%'))
    .select(['EMAIL', 'FECHA'])
)

redenciones_totales_GO199 = redenciones_cloud_GO199.union_all(redenciones_olo_GO199)

redenciones_octubre_GO199 = (
    redenciones_totales_GO199
    .join(tiempo, on = 'FECHA')
    .filter(fn.col('ANIO_ALSEA') == 2023)
    .filter(fn.col('MES_ALSEA') == 10)
    .with_column_renamed('FECHA', 'FECHA_ADQUISICION')
    .select('EMAIL', 'FECHA_ADQUISICION')
)

adquisiciones_octubre_GO199 = (
    redenciones_octubre_GO199
    .join(primeras_compras_octubre, on = ['FECHA_ADQUISICION', 'EMAIL'])
)

datos_recompras_octubre_GO199 = (
    adquisiciones_octubre_GO199
    .join(transacciones_dp_total, on = 'EMAIL')
    .filter(fn.col('FECHA') > fn.col('FECHA_ADQUISICION'))
    .group_by('EMAIL')
    .agg(
        fn.sum('VENTA').alias('VENTA'),
        fn.count_distinct('ORDER_ID').alias('TRANSACCIONES')
    )
    .with_column('COHORT',
        fn.when(fn.col('TRANSACCIONES') > 3, 4)
        .otherwise(fn.col('TRANSACCIONES'))
    )
    .group_by('COHORT')
    .agg(
        fn.count_distinct('EMAIL').alias('USUARIOS'),
        fn.sum('VENTA').alias('VENTA'),
        fn.sum('TRANSACCIONES').alias('TRANSACCIONES')
    )
    .with_column('TICKET', fn.col('VENTA') / fn.col('TRANSACCIONES'))
    .with_column('FRECUENCIA', fn.col('TRANSACCIONES') / fn.col('USUARIOS'))
)

datos_recompras_octubre_GO199_pd = datos_recompras_octubre_GO199.to_pandas()
datos_recompras_octubre_GO199_pd.to_csv('datos_recompras_octubre_GO199.csv')
datos_recompras_octubre_GO199_pd

,COHORT,USUARIOS,VENTA,TRANSACCIONES,TICKET,FRECUENCIA
0,1,1567,385038.663793,1567,245.717080,1.000000
1,4,53,58226.206897,266,218.895515,5.018868
2,2,328,153594.827586,656,234.138457,2.000000
3,3,88,63027.413793,264,238.740204,3.000000


# Datos Recompra Recuperacion

In [30]:
from DP_transacciones_total import get_transacciones_dp_total
from general_activos_en_serie_de_tiempo import get_activos_en_serie_de_tiempo
from general_primeras_compras import get_primeras_compras
from general_get_tiempo import get_tiempo

transacciones_dp_total = get_transacciones_dp_total(session)

tiempo = get_tiempo(session)

primeras_compras = (
    get_primeras_compras(transacciones_dp_total)
    .join(tiempo, on = 'FECHA')
    .select('EMAIL', 'ANIO_ALSEA', 'SEM_ALSEA')
    .with_column('PRIMERA_COMPRA', fn.lit(True))
)

activos_en_semana_actual = (
    get_activos_en_serie_de_tiempo(session, transacciones_dp_total, ['ANIO_ALSEA', 'SEM_ALSEA'])
    .select('EMAIL', 'ANIO_ALSEA', 'SEM_ALSEA')
    .distinct()
    .with_column('ACTIVO_EN_SEMANA_ACTUAL', fn.lit(True))
)

activos_en_semana_previa = (
    activos_en_semana_actual
    .with_column('ANIO_ALSEA', fn.when(fn.col('SEM_ALSEA') == 52, fn.col('ANIO_ALSEA') + 1).otherwise(fn.col('ANIO_ALSEA')))
    .with_column('SEM_ALSEA', fn.when(fn.col('SEM_ALSEA') == 52, 1).otherwise(fn.col('SEM_ALSEA') + 1))
    .select('EMAIL', 'ANIO_ALSEA', 'SEM_ALSEA')
    .with_column('ACTIVO_EN_SEMANA_PREVIA', fn.lit(True))
)

movimientos_base = (
    activos_en_semana_actual
    .join(activos_en_semana_previa, on = ['EMAIL', 'ANIO_ALSEA', 'SEM_ALSEA'], how = 'fullouter')
    .join(primeras_compras, on = ['EMAIL', 'ANIO_ALSEA', 'SEM_ALSEA'], how = 'left')
    .with_column(
        'TYPE',
        fn.when(fn.col('PRIMERA_COMPRA'), 'PRIMERAS_COMPRAS') 
        .when(fn.col('ACTIVO_EN_SEMANA_ACTUAL') & fn.col('ACTIVO_EN_SEMANA_PREVIA'), 'ACTIVO_EN_AMBAS_SEMANAS')
        .when(fn.col('ACTIVO_EN_SEMANA_ACTUAL'), 'RECUPERADOS')
        .when(fn.col('ACTIVO_EN_SEMANA_PREVIA'), 'RECIEN_INACTIVADOS')
    )
    .filter(fn.col('TYPE') != 'ACTIVOS_EN_AMBAS_SEMANAS')
)

In [31]:
tiempo = get_tiempo(session)

primera_compra_por_semana = (
    get_transacciones_dp_total(session)
    .join(tiempo, on = 'FECHA')
    .group_by('EMAIL', 'ANIO_ALSEA', 'SEM_ALSEA')
    .agg(fn.min('FECHA').alias('PRIMERA_COMPRA_EN_SEMANA'))
)

recuperados_octubre = (
    movimientos_base
    .filter(fn.col('ANIO_ALSEA') == 2023)
    .filter(fn.col('SEM_ALSEA').isin([40, 41, 42, 43]))
    .filter(fn.col('TYPE') == 'RECUPERADOS')
    .join(primera_compra_por_semana, on = ['EMAIL', 'ANIO_ALSEA', 'SEM_ALSEA'])
    .with_column_renamed('PRIMERA_COMPRA_EN_SEMANA', 'FECHA_RECUPERACION')
)

## Recuperados general

In [32]:
from DP_transacciones_total import get_transacciones_dp_total
from general_primeras_compras import get_primeras_compras
from general_get_tiempo import get_tiempo

tiempo = get_tiempo(session)

datos_recompras_octubre = (
    recuperados_octubre
    .join(transacciones_dp_total, on = 'EMAIL')
    .filter(fn.col('FECHA') > fn.col('FECHA_RECUPERACION'))
    .group_by('EMAIL')
    .agg(
        fn.sum('VENTA').alias('VENTA'),
        fn.count_distinct('ORDER_ID').alias('TRANSACCIONES')
    )
    .order_by('TRANSACCIONES', ascending = False)
    .with_column('COHORT',
        fn.when(fn.col('TRANSACCIONES') > 3, 4)
        .otherwise(fn.col('TRANSACCIONES'))
    )
    .group_by('COHORT')
    .agg(
        fn.count_distinct('EMAIL').alias('USUARIOS'),
        fn.sum('VENTA').alias('VENTA'),
        fn.sum('TRANSACCIONES').alias('TRANSACCIONES')
    )
    .with_column('TICKET', fn.col('VENTA') / fn.col('TRANSACCIONES'))
    .with_column('FRECUENCIA', fn.col('TRANSACCIONES') / fn.col('USUARIOS'))
)

datos_recompras_octubre_pd = datos_recompras_octubre.to_pandas()
datos_recompras_octubre_pd.to_csv('datos_recompras_octubre_recuperacion.csv')
datos_recompras_octubre_pd

,COHORT,USUARIOS,VENTA,TRANSACCIONES,TICKET,FRECUENCIA
0,1,9128,2.321178e+06,9128,254.292113,1.000000
1,4,168,1.834595e+05,803,228.467600,4.779762
2,2,1618,7.984217e+05,3236,246.731051,2.000000
3,3,380,2.619912e+05,1140,229.816848,3.000000


## Recuperados cupon 40% OFF

In [40]:
from DP_transacciones_total import get_transacciones_olo, get_transacciones_cloud

descuentos_40_OFF = (
    session
    .table('SEGMENT_EVENTS.ALOPERACIONES.TAFDESCUENTOSORDENES3')
    .filter(fn.col('FCIDDESCUENTO').isin([4501, 4576]))
    .select(['FCIDTIENDA', 'FITICKET'])
)

ordenes_40_OFF = (
    session
    .table('SEGMENT_EVENTS.ALOPERACIONES.TAFORDENES')
    .filter(fn.col('FIIDCLIENTE').is_not_null())
    .filter(fn.col('FIIDCLIENTE') != 0)
    .with_column('ID_CLIENTE', fn.concat(fn.col('FCIDTIENDA'), fn.col('FIIDCLIENTE')))
    .with_column('FECHA', fn.to_date('FDIDDIA'))
    .join(descuentos_40_OFF, on = ['FCIDTIENDA', 'FITICKET'], how = 'INNER')
)

clientes_40_OFF = (
    session
    .table('SEGMENT_EVENTS.ALMODCOMUN.TACCLIENTES')
    .with_column('ID_CLIENTE', fn.concat(fn.col('FIIDTIENDA'), fn.col('FIIDCLIENTE')))
    .join(ordenes_40_OFF, on = 'ID_CLIENTE')
)

transacciones_olo = get_transacciones_olo(session)

redenciones_olo_40_OFF = (
    transacciones_olo
    .join(clientes_40_OFF, on = ['ID_CLIENTE', 'FECHA'])
    .select(['EMAIL', 'FECHA'])
)

redenciones_cloud_40_OFF = (
    get_transacciones_cloud(session)
    .filter(fn.col('COUPONSCODE').like('%MISSU%'))
    .select(['EMAIL', 'FECHA'])
)

redenciones_totales_40_OFF = redenciones_cloud_40_OFF.union_all(redenciones_olo_40_OFF)

redenciones_octubre_40_OFF = (
    redenciones_totales_40_OFF
    .join(tiempo, on = 'FECHA')
    .filter(fn.col('ANIO_ALSEA') == 2023)
    .filter(fn.col('MES_ALSEA') == 10)
    .with_column_renamed('FECHA', 'FECHA_RECUPERACION')
    .select('EMAIL', 'FECHA_RECUPERACION')
)

redenciones_octubre_40_OFF.agg(fn.count_distinct('EMAIL')).show()

datos_recompras_octubre_40_OFF = (
    redenciones_octubre_40_OFF
    .join(transacciones_dp_total, on = 'EMAIL')
    .filter(fn.col('FECHA') >= fn.col('FECHA_RECUPERACION'))
    .group_by('EMAIL')
    .agg(
        fn.sum('VENTA').alias('VENTA'),
        fn.count_distinct('ORDER_ID').alias('TRANSACCIONES')
    )
    .order_by('TRANSACCIONES', ascending = False)
    .with_column('COHORT',
        fn.when(fn.col('TRANSACCIONES') > 3, 4)
        .otherwise(fn.col('TRANSACCIONES'))
    )
    .group_by('COHORT')
    .agg(
        fn.count_distinct('EMAIL').alias('USUARIOS'),
        fn.sum('VENTA').alias('VENTA'),
        fn.sum('TRANSACCIONES').alias('TRANSACCIONES')
    )
    .with_column('TICKET', fn.col('VENTA') / fn.col('TRANSACCIONES'))
    .with_column('FRECUENCIA', fn.col('TRANSACCIONES') / fn.col('USUARIOS'))
)

datos_recompras_octubre_40_OFF_pd = datos_recompras_octubre_40_OFF.to_pandas()
datos_recompras_octubre_40_OFF_pd.to_csv('datos_recompras_octubre_40_OFF_recuperacion.csv')
datos_recompras_octubre_40_OFF_pd

---------------------------
|"COUNT(DISTINCT EMAIL)"  |
---------------------------
|976                      |
---------------------------



,COHORT,USUARIOS,VENTA,TRANSACCIONES,TICKET,FRECUENCIA
0,1,808,233239.137931,808,288.662299,1.000000
1,4,10,24561.034483,50,491.220690,5.000000
2,3,21,19214.655172,63,304.994527,3.000000
3,2,137,96890.000000,274,353.613139,2.000000
